In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
cap = cv2.VideoCapture("vid1_IR.avi")


def filter_labels(stats):
    res = []
    stats = [x for x in stats if x[2] < x[3]]
    added = []

    WIDTH_DIFF = 25
    for i in range(len(stats)):
        x_1, y_1, width_1, height_1, _ = stats[i]

        for j in range(i +1, len(stats)):
            x_2, y_2, width_2 ,height_2, _ = stats[j]

            if x_1 - WIDTH_DIFF < x_2 and x_1 + width_1 + WIDTH_DIFF > x_2:
                res_x = x_1
                res_y = y_1
                res_w = width_1
                res_h = height_1 + height_2
                res.append((res_x, res_y, res_w, res_h))
                added.append(j)

        if not i in added:
            res.append((x_1, y_1, width_1, height_1))

    for i in range(len(stats)):
        if i not in added:
            # print(stats[i][4])
            if stats[i][4] > 3000:
                res.append((stats[i][0], stats[i][1], stats[i][2], stats[i][3]))

    return res

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    G = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    ret, G_bin = cv2.threshold(G,35,255,cv2.THRESH_BINARY)
    
    n_size = 7

    ret, G_bin = cv2.threshold(G,35,255,cv2.THRESH_BINARY)

    G_bin = cv2.erode(G_bin, np.ones((n_size, n_size)))
    G_bin = cv2.dilate(G_bin, np.ones((n_size, n_size)))
    G_bin = cv2.dilate(G_bin, np.ones((5,5)))

    output_image = cv2.cvtColor(G_bin, cv2.COLOR_GRAY2BGR)

    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(G_bin, connectivity=8)

    # filtered_labels = filter_labels(stats[1:])

    # for i in range(len(filtered_labels)):
    #     x, y, w, h = filtered_labels[i]

    #     cv2.rectangle(output_image, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow("IR", output_image)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break


cv2.destroyAllWindows()
cap.release()

In [4]:

def get_distance(box1, box2):
    x1_min, y1_min, x1_max, y1_max = box1
    x2_min, y2_min, x2_max, y2_max = box2

    if x1_max < x2_min:  # box1 is to the left of box2
        x_distance = x2_min - x1_max
    elif x2_max < x1_min:  # box2 is to the left of box1
        x_distance = x1_min - x2_max
    else:  # boxes overlap in the x direction
        x_distance = 0

    if y1_max < y2_min:  # box1 is above box2
        y_distance = y2_min - y1_max
    elif y2_max < y1_min:  # box2 is above box1
        y_distance = y1_min - y2_max
    else:  # boxes overlap in the y direction
        y_distance = 0

    return math.sqrt(x_distance ** 2 + y_distance ** 2)

def merge_boxes(boxes):
    x_min = min(box[0] for box in boxes)
    y_min = min(box[1] for box in boxes)
    x_max = max(box[2] for box in boxes)
    y_max = max(box[3] for box in boxes)
    return [x_min, y_min, x_max, y_max]

def merge_nearby_bounding_boxes(bounding_boxes, threshold):
    merged = True
    while merged:
        merged = False
        to_merge = set()
        n = len(bounding_boxes)

        # Mark boxes to merge
        for i in range(n):
            for j in range(i + 1, n):
                if get_distance(bounding_boxes[i], bounding_boxes[j]) <= threshold:
                    to_merge.add(i)
                    to_merge.add(j)

        # Create new list of bounding boxes by merging the marked ones
        new_bounding_boxes = []
        skip = set()
        for i in range(n):
            if i in skip:
                continue
            if i in to_merge:
                group = [bounding_boxes[i]]
                for j in range(i + 1, n):
                    if j in to_merge and get_distance(bounding_boxes[i], bounding_boxes[j]) <= threshold:
                        group.append(bounding_boxes[j])
                        skip.add(j)
                new_bounding_boxes.append(merge_boxes(group))
                merged = True
            else:
                new_bounding_boxes.append(bounding_boxes[i])

        bounding_boxes = new_bounding_boxes

    return bounding_boxes


In [8]:
cap = cv2.VideoCapture("vid1_IR.avi")
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    G = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    ret, G_bin = cv2.threshold(G,35,255,cv2.THRESH_BINARY)
    
    n_size = 7

    ret, G_bin = cv2.threshold(G,35,255,cv2.THRESH_BINARY)

    G_bin = cv2.erode(G_bin, np.ones((n_size, n_size)))
    G_bin = cv2.dilate(G_bin, np.ones((n_size, n_size)))
    G_bin = cv2.dilate(G_bin, np.ones((5,5)))

    output_image = cv2.cvtColor(G_bin, cv2.COLOR_GRAY2BGR)

    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(G_bin, connectivity=8)

    boxes = stats[1:]
    boxes = [(x,y,x+w,y+h) for x,y,w,h,_ in boxes]
    
    filtered_labels = merge_nearby_bounding_boxes(boxes, 5)
    
    filtered_labels = [(x,y,w,h) for x,y,w,h in filtered_labels]

    for i in range(len(filtered_labels)):
        x, y, w, h = filtered_labels[i]
        a_w = w - x
        a_h = h - y
        if a_h > a_w:
            cv2.rectangle(output_image, (x, y), (x + a_w, y + a_h), (0, 255, 0), 2)

    cv2.imshow("IR", output_image)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cv2.destroyAllWindows()
cap.release()